=======================================================================
# **STEM AWAY Project** I [Mac Power Users](https://talk.macpowerusers.com/c/episodes/6) Forum
=======================================================================

### Machine learning Pathway

#### Prepared by [Sunlong Ngouv](https://github.com/SunlongNgouv) on 08-07-2020

#Web Scrapping

In [ ]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install transformers
!pip install HTMLParser

     |████████████████████████████████| 133kB 4.3MB/s 
     |████████████████████████████████| 92kB 5.0MB/s 
     |████████████████████████████████| 778kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 12.8MB/s 
     |████████████████████████████████| 890kB 19.6MB/s 
     |████████████████████████████████| 3.0MB 39.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=94c7ac5801f092529a827411049c45d3ff2fd4928dd5b341455e593ab3848e48
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for HTMLParser: filename=HTMLParser-0.0.2-cp36-none-any.whl size=5983 sha256=350ab7939962def8b477128f6e3c7c2c37d01e293283b29da7ebc34175a739c3
  Stored in directory: /root/.cache/pip/wheels/e0/a1/d9/1655169e4c4354903ca3adc524b2cfd60d44767144c1faef62
Successfully built HTMLParser


In [ ]:
# Libraries for web scrapping
import requests
from lxml import html
from bs4 import BeautifulSoup
import re
import pandas as pd

# Libraries for data preprocessing
import re
import nltk
nltk.download('all')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from html.parser import HTMLParser

# Libraries for BERT
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [ ]:
# upload data from url to notebook
r = requests.get('https://talk.macpowerusers.com/c/episodes/6')

In [ ]:
# perse html
soup = BeautifulSoup(r.text,'html.parser')

In [ ]:
# extract parts in html with tags
results = soup.find_all('tr', class_='topic-list-item')

In [ ]:
len(results)

30

In [ ]:
# extract tags
response = []
for i in range (0,len(results)):
  re_num = results[i]

  topic = re.sub(r'\n', '',re_num.find('span').text)
  replies = re.sub(r'\n', '', re_num.find('td', attrs={'class':'replies'}).text)
  view = re.sub(r'\n', '', re_num.find('td', attrs={'class':'views'}).text)
  latest_update = re.sub('\s+',' ',re_num.contents[9].text)
  poster = re_num.find_all('a')[1]['href']
  episode = re_num.find('a')['href']

  # get all urls in the page
  urls = [] 
  urls.append(re_num.find('span',class_='link-top-line').a.get('href'))

  # get comment/feedback in each url
  for b in range(0,len(urls)):
    links = urls[b]
    rr = requests.get(links)
    data = BeautifulSoup(rr.text, 'html.parser')
    comment = data.find_all('p')

  # convert comments to texts
  comments = []
  for n in range(0, len(comment)):
    comments.append(re.sub(r'\n','', comment[n].text.strip()))

  # get poster names
  authors = []
  for a in data.find_all('span', class_='creator'):
    for b in a:
      for c in b:
        for d in c:
          x = re.sub(r'\n','',d)
          if x == '':
            break
          else:
            x
          authors.append(x)

  # add all tags together
  response.append((topic,replies,view, latest_update, poster, episode, authors, comments))

print(response)

[('About the Episodes category', '1', '365', ' June 15, 2018 ', 'https://talk.macpowerusers.com/u/mpubot', 'https://talk.macpowerusers.com/t/about-the-episodes-category/17', ['mpubot'], ['Discussions about the Mac Power Users podcast episodes, hosted by @MacSparky and @ismh.', 'Powered by Discourse, best viewed with JavaScript enabled']), ('548: Jumping Into Markdown', '94', '1496', ' August 24, 2020 ', 'https://talk.macpowerusers.com/u/mpubot', 'https://talk.macpowerusers.com/t/548-jumping-into-markdown/18679', ['mpubot', 'drdrang', 'MitchWagner', 'CFunk', 'MotownDoc', 'rlivingston', 'rlivingston', 'tjluoma', 'lsamberg', 'creighton', 'ismh', 'RogerDowning', 'rlamarch', 'bowline', 'Brad', 'bowline', 'TimM', 'Brad', 'bowline', 'sdevore'], ["From jotting quick notes to writing for the web, Markdown has become a very popular choice for formatting plain text. It's syntax is human-readable and easy to learn. This week, David and Stephen teach Markdown 101 and more.", 'I remember the first M

In [ ]:
# rename the column names
title = ['TOPIC', 'REPLY', 'VIEW', 'LATEST UPDATE', 'POSTER', 'EPISODE URL', 'Authors', 'COMMENT']

# put data into dataframe
df = pd.DataFrame(response, columns=title)
df

,TOPIC,REPLY,VIEW,LATEST UPDATE,POSTER,EPISODE URL,Authors,COMMENT
0,About the Episodes category,1,365,"June 15, 2018",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/about-the-epi...,[mpubot],[Discussions about the Mac Power Users podcast...
1,548: Jumping Into Markdown,94,1496,"August 24, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/548-jumping-i...,"[mpubot, drdrang, MitchWagner, CFunk, MotownDo...",[From jotting quick notes to writing for the w...
2,550: The World of RSS,3,75,"August 24, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/550-the-world...,"[mpubot, tjluoma, D_Rehak]","[RSS can bring news, blog posts and even podca..."
3,546: The Best Listeners,26,667,"August 23, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/546-the-best-...,"[mpubot, MereCivilian, drhcc, tjluoma, cornchi...","[On this feedback show, David and Stephen go t..."
4,549: Brett Terpstra's New Robe,20,734,"August 21, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/549-brett-ter...,"[mpubot, bowline, JohnAtl, tjluoma, vgrovpro, ...",[Brett Terpstra returns to the show to talk ab...
5,544: The Notes Roundup,87,2651,"August 21, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/544-the-notes...,"[mpubot, brettwignall, bowline, ismh, MitchWag...",[The market for note taking applications is hu...
6,"547: Of a Certain Age, with Jim Metzendorf",5,334,"August 16, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/547-of-a-cert...,"[mpubot, Benjamin_D, bowline, mattduffy941, co...","[This week, the editor of Mac Power Users, Jim..."
7,542: Beta Tales,63,1070,"August 7, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/542-beta-tale...,"[mpubot, ismh, professorgadgets, MitchWagner, ...","[It's beta season, and David and Stephen have ..."
8,545: State of the Apple Watch,25,529,"August 2, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/545-state-of-...,"[mpubot, MitchWagner, missionpk1, tjluoma, cor...",[The Apple Watch has undergone massive changes...
9,537: The Photos Field Guide and Apple Photos,39,1099,"July 27, 2020",https://talk.macpowerusers.com/u/mpubot,https://talk.macpowerusers.com/t/537-the-photo...,"[mpubot, sgclark, solid0270, sgclark, swcolema...","[From the early days of iPhoto to now, with Ph..."


#Data Preprocessing

In [ ]:
df['COMMENT'][1]

["From jotting quick notes to writing for the web, Markdown has become a very popular choice for formatting plain text. It's syntax is human-readable and easy to learn. This week, David and Stephen teach Markdown 101 and more.",
 'I remember the first MPU Markdown episode fondly. David got a lot wrong, but his enthusiasm for the format was undeniable.',
 'Good tip on using code blocks for annotation. I’m looking forward to giving that a try.',
 'This episode has me looking back into Markdown and where it fits in my workflows. I did a deep dive about 5 years ago but it never really stuck. A lot has changed since then, so it’s worth taking another look.Thanks!',
 'Okay, I know I’m going to sound like an idiot here, but I just finished listening to that episode, downloaded Byword, and typed up some sample text using Markdown. What I got was a lot of asterisks, underscores, backwards quotes, etc., but I’m not seeing the FORMATTED text (like the bold, italics, code blocks, etc.). What am I 

In [ ]:
# remove punctuation and space

def pre_process(text):
    #convert HTML character codes to ASCII code
    parser = HTMLParser()
    text = parser.unescape(text)
    
    text = text.strip()     # remove all leading and trailing spaces from the strings
    text = re.sub(r'<[^>]*>', '',text) # remove HTML tags
    text = re.sub(r'(?:\#+[\w_]+[\w\'_\]*[\w_]+)', '', text) # remove hashtags
    text = re.sub('\n', ' ', text)  # remove new line
    text = re.sub('@', '', text)    # remove @ sign
    text=re.sub("(\\d|\\W)+"," ",text)    # remove special characters and digits
    text=text.lower()       # lowercase
    return text

df['text'] = df['COMMENT'].apply(lambda x:pre_process(x))

TypeError: ignored

In [ ]:
from google.colab import files
data.to_csv('mac power user -- web scraping.csv') 
files.download('mac power users -- web scraping.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>